<a href="https://colab.research.google.com/github/Erocr/fairness/blob/main/UPP26/TD2_mepsdata_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Préambule : nos biais inconscients

Nous vous proposons, si vous le souhaitez, de prendre une dizaine de minutes pour tester vos biais inconscients:

https://implicit.harvard.edu/implicit/canadafr/takeatest.html


# TD 2: Manipulation des données

## Objectives


 1. Study the data, the distribution of each feature and its relation to the target.

 2. Highlight some bias present in the data


## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings ---- for Colab Users ONLY
  The next two cells of code are too execute only once per colab environment


#### 1. Python env creation (Colab only)

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 2. Download MEPS dataset (for part2) it can take several minutes (Colab only)

        ```
        ! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
        ! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ```

  
### Local Settings ---- for installation on local computer ONLY

You can use the same env as TD1, you will then need to follow steps 2 and 4

#### 1. Uv installation (local only, no need to redo if already done)


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`

#### 2. R installation *NEW* (local only)

        In the command `Rscript` says 'command not found'

        `sudo apt install r-base-core`

#### 3. Python env creation (local only, no need to redo if already done)

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv venv
        uv pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 4. Download MEPS dataset, it can take several minutes *NEW* (local only)

        ```
        cd TD_bias_mitigation/.venv/lib/python3.12/site-packages/aif360/data/raw/meps/
        Rscript generate_data.R
        ```

## Dataset: Meps

We recommend consulting the following pages for a better understanding of the dataset: [MEPSDataset19](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.MEPSDataset19.html) and the [AIF360 tutorial](https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb)

What you need to have read
- **The sensitive attribute is 'RACE' :1 is privileged, 0 is unprivileged** ; It is constructed as follows: 'Whites' (privileged class) defined by the features RACEV2X = 1 (White) and HISPANX = 2 (non Hispanic); 'Non-Whites' that included everyone else.
(The features 'RACEV2X', 'HISPANX' etc are removed, and replaced by the 'RACE')
- **'UTILIZATION' is the outcome (the label to predict for a ML model) 0 is positive 1 is negative**. It is a binary composite feature, created to measure the total number of trips requiring some sort of medical care, it sum up the following features (that are removed from the data):
    * OBTOTV15(16), the number of office based visits
    * OPTOTV15(16), the number of outpatient visits
    * ERTOT15(16), the number of ER visits
    * IPNGTD15(16), the number of inpatient nights
    * HHTOTD16, the number of home health visits
UTILISATION is set to 1 when te sum is above or equal to 10, else it is set to 0
- **The dataset is weighted** The dataset come with an 'instance_weights' attribute that corresponds to the feature perwt15f these weights are supposed to generate estimates that are representative of the United State (US) population in 2015.


Summary to remember
- **The sensitive attribute is 'RACE' :1 is privileged, 0 is unprivileged**
- **'UTILIZATION' is the outcome (the label to predict for a ML model) 0 is positive 1 is negative**
- **The dataset is weighted**


In [ ]:
# To execute only in Colab
! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit

In [3]:
! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/


By using this script you acknowledge the responsibility for reading and
abiding by any copyright/usage rules and restrictions as stated on the
MEPS web site (https://meps.ahrq.gov/data_stats/data_use.jsp).

Continue [y/n]? > y
Loading required package: foreign
trying URL 'https://meps.ahrq.gov/mepsweb/data_files/pufs/h181ssp.zip'
Content type 'application/zip' length 13303652 bytes (12.7 MB)
downloaded 12.7 MB

Loading dataframe from file: h181.ssp
Exporting dataframe to file: h181.csv
trying URL 'https://meps.ahrq.gov/mepsweb/data_files/pufs/h192ssp.zip'
Content type 'application/zip' length 15505898 bytes (14.8 MB)
downloaded 14.8 MB

Loading dataframe from file: h192.ssp
Exporting dataframe to file: h192.csv


In [4]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In this TD we will use data from the [Medical Expenditure Panel Survey](https://meps.ahrq.gov/mepsweb/). The TD is inspired from [AIF360 tutorial](https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_medical_expenditure.ipynb)

## The Meps dataset

In [5]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', append=True, category=UserWarning)


In [6]:
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21

MEPSDataset19_data = MEPSDataset19()
# (dataset_orig_panel19_train,
#  dataset_orig_panel19_val,
#  dataset_orig_panel19_test) = MEPSDataset19_data.split([0.5, 0.8], shuffle=True)

In [7]:
(dataset_orig_panel19_train,
 dataset_orig_panel19_val,
 dataset_orig_panel19_test) = MEPSDataset19().split([0.5, 0.8], shuffle=True)

In [8]:
len(dataset_orig_panel19_train.instance_weights), len(dataset_orig_panel19_val.instance_weights), len(dataset_orig_panel19_test.instance_weights)

(7915, 4749, 3166)

In [9]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights


array([21854.981705, 18169.604822, 17191.832515, ...,  3896.116219,
        4883.851005,  6630.588948])

In [10]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

'Taille du dataset 15830, poids total du dataset 141367240.546316.'

### Premier appercu du dataset

La librairie AIF360 fournie une surcouche au dataset, cela le rend un peu moins intuitif d'utilisation (par exemple pour étudier/visualiser les attributs un à un), mais elle permet de calculer les métrique des fairness en une ligne de commande.

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

metric_orig_panel19_train = BinaryLabelDatasetMetric(
        MEPSDataset19_data,
        unprivileged_groups=[{'RACE': 0}],
        privileged_groups=[{'RACE': 1}])

print(metric_orig_panel19_train.disparate_impact())

Cependant le but de ce TD étant encore de manipuler les données et de les analyser nous allons revenir aux données sous forme d'un dataframe.

Note pour calculer les métriques de fairness sans avoir à les réimplémenter dans le cas pondéré (instances weights) vous pouvez utiliser les méthodes implémenter dans AIF360 là [Implémentation Métriques de Fairness](https://aif360.readthedocs.io/en/latest/modules/sklearn.html#module-aif360.sklearn.metrics)

### Conversion en un dataframe

Nous avons vu que la somme des poids est conséquente, pres de 115millions nous ne pouvons donc pas raisonneblement dupliqué chaque ligne autant de fois que son poids.

Nous allons stocker la pondération et la prendre en compte ensuite dans notre analyse

In [11]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df['WEIGHT'] = data[1]['instance_weights']
    return df

df = get_df(MEPSDataset19_data)



In [12]:
from aif360.sklearn.metrics import disparate_impact_ratio, base_rate
dir = disparate_impact_ratio(
    y_true=df.UTILIZATION,
    prot_attr=df.RACE,
    pos_label=0,
    sample_weight=df.WEIGHT)
br =base_rate(
    y_true=df.UTILIZATION,
    pos_label=0,
    sample_weight=df.WEIGHT)
dir,br

(1.1848351529675123, np.float64(0.7849286063696154))

In [13]:
dir = disparate_impact_ratio(
    y_true=df.UTILIZATION,
    prot_attr=df.RACE,
    pos_label=0)
br =base_rate(
    y_true=df.UTILIZATION,
    pos_label=0)
dir,br

(1.1746792888264614, np.float64(0.8283006948831333))

## Question 1 - Apprendre un modèle pour prédire le fait d'être réadmis
### 1.1 - Faire le pre-processing des données

Ici ce pre-processing a déjà été fait par AIF, nous avons simplement converti le dataset en dataframe pour pouvoir le manipuler librement

### Question 1.2 - Creer les échantillons d'apprentissage, de validation et de test

Pour créer le df_X il faut enlever l'outcome ("UTILIZATION") et la pondération ("WEIGHT")

La colonne "UTILIZATION" sera le label (noté y)

La colonne "WEIGHT" sera la pondération (notée w)


In [21]:
df_X = df.drop(columns=["UTILIZATION", "WEIGHT"])
y = df["UTILIZATION"]
w = df["WEIGHT"]

In [50]:
I = np.arange(len(df))
np.random.shuffle(I)

train_size = 0.6
val_size = 0.2
test_size = 0.2

assert train_size + val_size + test_size == 1, "Faut que la somme fasse 1"

I_train = I[:int(train_size*len(df))]
I_val = I[int(train_size*len(df)):int((train_size+val_size)*len(df))]
I_test = I[int((train_size+val_size)*len(df)):]

X_train = df_X.iloc[I_train]
y_train = y.iloc[I_train]
w_train = w.iloc[I_train]
X_val = df_X.iloc[I_val]
y_val = y.iloc[I_val]
w_val = w.iloc[I_val]
X_test = df_X.iloc[I_test]
y_test = y.iloc[I_test]
w_test = w.iloc[I_test]

In [44]:
df_X.shape, X_train.shape, y_train.shape, w_train.shape, X_val.shape, y_val.shape, w_val.shape,  X_test.shape, y_test.shape, w_test.shape

((15830, 138),
 (9498, 138),
 (9498,),
 (9498,),
 (3166, 138),
 (3166,),
 (3166,),
 (3166, 138),
 (3166,),
 (3166,))

### Question 1.3 - Apprendre une regression logistique dont le but est de prédire UTILIZATION

In [35]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression()
logisticRegression.fit(X_train, y_train, sample_weight=w_train)

LogisticRegression()

### Quesiton 1.4 Performance du modèle (afficher la matrice de confusion)

In [36]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, logisticRegression.predict(X_test))

In [42]:

px.imshow(cm, text_auto=True)

### Question 1.5 - Calculer les métriques "group fairness" du modèle

In [62]:
y_pred = logisticRegression.predict(X_test)
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [63]:
y_pred.shape

(3166,)

In [68]:
group_metrics = get_metrics(y_train,
                            y_pred=logisticRegression.predict(X_train),
                            pos_label=0,
                            prot_attr=X_train["RACE"],
                            sample_weight=w_train)

In [69]:
group_metrics

{'base_rate_truth': np.float64(0.7913269087147874),
 'statistical_parity_difference': np.float64(0.12211109122568164),
 'disparate_impact_ratio': 1.1490584642258694,
 'base_rate_preds': np.float64(0.868664239144242),
 'equal_opportunity_difference': 0.04740060299456239,
 'average_odds_difference': 0.13125459863041428,
 'conditional_demographic_disparity': np.float64(0.04903231908627063),
 'smoothed_edf': np.float64(1.1253256192424148),
 'df_bias_amplification': np.float64(0.488400537162083),
 'balanced_accuracy_score': np.float64(0.6953718359414705)}

## Question 2 - Etude de l'impact de la couleur de peau sur le faire d'être réadmis, les prédictions du modèle et ses liens avec les autres variables


### Question 2.1 - Faire l'étude descriptive univarié de la couleur de peau ('RACE') (effectif, fréquence, model)

In [72]:
s_train = X_train["RACE"]
s_val = X_val["RACE"]
s_test = X_test["RACE"]

In [84]:
N = len(s_train)
K = int(s_train.sum())
print(f"effectif: {K}/{N-K}")
print(f"frequence: {K/N}")
mode = 0 if K < N/2 else 1
print(f"mode: {mode}")
print("model: ?")

nb_priv_race = N-K
nb_prot_race = K

effectif: 3358/6140
frequence: 0.35354811539271425
mode: 0
model: ?


### Question 2.2 - Faire des graphiques décrivant la couleur de peau  (diagramme en secteur, diagramme en barres)

In [86]:
px.bar([nb_priv_race, nb_prot_race])

In [87]:
px.pie([nb_priv_race, nb_prot_race])

### Question 2.3 - Faire l'analyse bivariée entre la couleur de peau et les autres variables explicatives quantitatives (boite à moustaches des variables par genre, densité/histogramme par genre, rapport de corrélation)

In [98]:
px.box(df, "RACE", "SEX=1")

In [100]:
px.histogram(df, x="RACE", y="SEX=1")

### Question 2.4 - Faire l'analyse bivariée entre la couelur de peau et d'autres variables explicatives qualitative (table de contingence, diagramme en barre selon les profils lignes et selon les profils colonnes, diagramme en mosaique)

In [ ]:
print("TODO")

### Question 2.5 - Faire l'analyse bivariée entre la couleur de peau et la colonne 'UTILIZATION'

In [ ]:
print("TODO")

### Question 2.6 - Faire l'analyse bivariée entre la couleur de peau et les prédictions du modèle prédisant la colonne 'UTILIZATION'

In [ ]:
print("TODO")

### Question 2.7 - Faire l'analyse bivariée entre la couleur de peau et les erreurs du modèle précédent

In [ ]:
print("TODO")

### Question 2.8 - Proposer un modèle à base d'une foret aléatoire prédisant la couleur de peau en fonction des autres variables explicatives

In [ ]:
print("TODO")

### Question 2.9 - Apprendre un modèle privé de la couleur de peau pour prédire UTILIZATION et étudier le lien entre ses prédictions et la couleur de peau

In [ ]:
print("TODO")

## Question 3 - Faire de meme pour une autre variable sensible (Age, genre, marry etc)


In [ ]:
print("TODO")

## Question 4 - Causalité: appliquer le module causal-learn sur le jeu de données en testant plusieurs des méthodes implémentées et en comparant les résultats entre les méthodes

In [ ]:
print("TODO")